# Problema

Predecir el coste del seguro

# El set de datos

* age: age of primary beneficiary

* sex: insurance contractor gender, female, male

* bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height,
objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9

* children: Number of children covered by health insurance / Number of dependents

* smoker: Smoking

* region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.

* charges: Individual medical costs billed by health insurance



In [3]:
# imports
import pandas as pd
from google.colab import drive


In [4]:
ruta = "insurance.csv"
data = pd.read_csv(ruta)

In [5]:
print(data.shape)
data.head()

(1338, 7)


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


# Objetivo

Generar un model de regresión capaz de predecir el valor del seguro en base a las características del cliente.

* Aplicar las técnicas oportunas de procesamiento de datos

* Valorar diferentes modelos de regresión

* Comparación entre modelos

* Ensemble

* Métricas

* Conclusiones finales


## Implementación

In [6]:
data.drop_duplicates()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


## comenzamos

In [7]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Definiendo las columnas categóricas y numéricas
categorical_columns = ['sex', 'smoker', 'region']
numerical_columns = ['age', 'bmi', 'children']

# Creando transformadores para las columnas categóricas y numéricas
categorical_transformer = OneHotEncoder(drop='first')  # Usamos drop='first' para evitar la multicolinealidad
numerical_transformer = StandardScaler()

# Combinando los transformadores en un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# Aplicando el preprocesamiento
X = data.drop('charges', axis=1)
y = data['charges']
X_preprocessed = preprocessor.fit_transform(X)

# Mostrando el tamaño del dataset preprocesado para verificar la transformación
X_preprocessed.shape


(1338, 8)

El preprocesamiento de datos ha sido exitoso, y ahora tenemos un total de 8 características en nuestro conjunto de datos preprocesado, resultado de la codificación One-Hot de las variables categóricas y la estandarización de las variables numéricas. Esto indica que las variables categóricas sex, smoker, y region han sido correctamente transformadas, junto con la normalización de age, bmi, y children.

Con los datos listos, el siguiente paso es seleccionar y evaluar diferentes modelos de regresión para predecir el costo del seguro. Vamos a comenzar con algunos modelos básicos y luego proceder a modelos más complejos, evaluando su rendimiento mediante validación cruzada. Los modelos a considerar serán Regresión Lineal, Árboles de Decisión, y Random Forest como punto de partida.

### Regresión lineal

La evaluación del modelo de Regresión Lineal mediante validación cruzada, utilizando el RMSE (Root Mean Squared Error) como métrica, arroja los siguientes resultados:

RMSE medio: $6,063.64

Desviación estándar del RMSE: $471.64

Esto indica que, en promedio, el modelo de Regresión Lineal se desvía aproximadamente 6,063.64 del valor real de los costos del seguro, con una variabilidad de 471.64 en las diferentes particiones del proceso de validación cruzada. Este es nuestro punto de referencia inicial para la precisión de la predicción.

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
import numpy as np

# Instanciando el modelo de Regresión Lineal
linear_model = LinearRegression()

# Creando un pipeline con el preprocesador y el modelo de regresión lineal
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', linear_model)])

# Evaluando el modelo mediante validación cruzada utilizando el RMSE como métrica
cv_scores = cross_val_score(pipeline, X, y, cv=10, scoring='neg_root_mean_squared_error')

# Calculando la media y la desviación estándar de los puntajes de validación cruzada
cv_scores_mean = np.mean(cv_scores)
cv_scores_std = np.std(cv_scores)

cv_scores_mean, cv_scores_std


(-6063.643882478561, 471.64418087941436)

###  Arbol de decision

In [9]:
from sklearn.tree import DecisionTreeRegressor

# Instanciando el modelo de Árbol de Decisión
decision_tree_model = DecisionTreeRegressor()

# Creando un pipeline con el preprocesador y el modelo de árbol de decisión
pipeline_dt = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', decision_tree_model)])

# Evaluando el modelo mediante validación cruzada utilizando el RMSE como métrica
cv_scores_dt = cross_val_score(pipeline_dt, X, y, cv=10, scoring='neg_root_mean_squared_error')

# Calculando la media y la desviación estándar de los puntajes de validación cruzada para el árbol de decisión
cv_scores_dt_mean = np.mean(cv_scores_dt)
cv_scores_dt_std = np.std(cv_scores_dt)

cv_scores_dt_mean, cv_scores_dt_std


(-6548.0398135346695, 490.53767640999496)

La evaluación del modelo de Árbol de Decisión mediante validación cruzada, utilizando el RMSE como métrica, proporciona los siguientes resultados:

RMSE medio: $6,525.16

Desviación estándar del RMSE: $484.10

Comparado con el modelo de Regresión Lineal, el Árbol de Decisión tiene un RMSE medio ligeramente mayor, lo que sugiere que este modelo, en su configuración actual, predice los costos del seguro con menos precisión. La desviación estándar del RMSE es similar, indicando una variabilidad comparable en las diferentes particiones de validación cruzada.

### Random forest


In [10]:
from sklearn.ensemble import RandomForestRegressor

# Instanciando el modelo de Random Forest
random_forest_model = RandomForestRegressor()

# Creando un pipeline con el preprocesador y el modelo de Random Forest
pipeline_rf = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', random_forest_model)])

# Evaluando el modelo mediante validación cruzada utilizando el RMSE como métrica
cv_scores_rf = cross_val_score(pipeline_rf, X, y, cv=10, scoring='neg_root_mean_squared_error')

# Calculando la media y la desviación estándar de los puntajes de validación cruzada para Random Forest
cv_scores_rf_mean = np.mean(cv_scores_rf)
cv_scores_rf_std = np.std(cv_scores_rf)

cv_scores_rf_mean, cv_scores_rf_std


(-4842.507579529709, 494.7609655560303)

La evaluación del modelo de Random Forest mediante validación cruzada, utilizando el RMSE como métrica, arroja los siguientes resultados:

RMSE medio: 4,833.16
Desviación estándar del RMSE: 535.34
Este resultado indica que el modelo de Random Forest mejora significativamente la precisión de la predicción del costo del seguro en comparación con los modelos de Regresión Lineal y Árbol de Decisión previamente evaluados. El RMSE medio más bajo sugiere que las predicciones de Random Forest están, en promedio, más cerca de los valores reales de los costos del seguro.

Con estos resultados, podemos concluir que el Random Forest es el modelo más prometedor entre los que hemos probado hasta ahora para este conjunto de datos. Podríamos explorar la optimización de hiperparámetros para este modelo para mejorar aún más su rendimiento.